In [1]:
#multi_agent.ipynb
#
#by Joe Hahn
#jmh.datasciences@gmail.com
#12 February 2018
#
#This uses Q-learning on multiple agents to demonstrate something...

In [2]:
#game settings
N_agents = 3                #number of agents
N_buckets = 5               #number of buckets
max_moves = 100             #max number of moves in single game
rn_seed = 14                #seed for random number generator

In [ ]:
#import game
from multi_agent import *
import time
time_start = time.time()

environment =  {'acts': ['move to 0', 'move to 1', 'move to 2', 'move to 3', 'move to 4'], 'actions': [0, 1, 2, 3, 4], 'N_agents': 3, 'max_moves': 100, 'rn_seed': 14, 'N_buckets': 5}
state =  {'agents': [{'bucket': 2}, {'bucket': 3}, {'bucket': 4}], 'next_agent': 1}
reward =  9.0
state_vector =  [[ 0.  0.  1.  1.  1.]]


Using TensorFlow backend.
